In [1]:
import pandas as pd
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn import metrics   #Additional scklearn functions
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import math
from IPython.display import display
import os


X_train = pd.read_csv("train_test_data/X_train.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
X_test = pd.read_csv("train_test_data/X_test.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
y_test = pd.read_csv("train_test_data/y_test.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
y_train = pd.read_csv("train_test_data/y_train.csv", index_col="uid").drop("Unnamed: 0", axis="columns")

In [2]:
# We only consider the post scores
class_names = [col for col in y_test.columns if "post" in col]
# Use the wk 9 to 10 training data only
used_weeks = [str(wk) for wk in range(9,11)]
filtered_columns = X_train.columns[X_train.columns.str.endswith(tuple(used_weeks))]
# Select all wk 9 and 10 features
filtered_features = X_train[filtered_columns]
train_data = {"class": {}, "regr": {}}
test_data = {"class": {}, "regr": {}}
# Prepare training and test data for classification model and regression model respectively
for name in class_names:
    # Classification labels
    if "class" in name:
        target_type = "class"
    else:
        target_type = "regr"
    train_data[target_type][name] = {"x": {}, "y": {}}
    test_data[target_type][name] = {"x": {}, "y": {}}
    # Only consider data which has the post score being not null
    not_null_indices = np.logical_not(y_train.loc[:, name].isnull())
    train_data[target_type][name]["x"] = filtered_features[not_null_indices]
    train_data[target_type][name]["y"] = y_train[not_null_indices].loc[:, name]
    test_data[target_type][name]["x"] = X_test[filtered_columns]
    test_data[target_type][name]["y"] = y_test.loc[:, name]

In [3]:
# Function to find the best hyperparameters for each model
def find_best_model(rscv, x, y):
    best_model = rscv.fit(x, y)
    print("processing")
    # Print out the best parameters for the model
    print()
    print("Optimal parameters: {", end="")
    print("Learning rate: ", best_model.best_estimator_.get_params()["learning_rate"], end=", ")
    print("Gamma: ", best_model.best_estimator_.get_params()["gamma"], end=", ")
    print("Max depth: ", best_model.best_estimator_.get_params()["max_depth"], end=", ")
    print("Subsample: ", best_model.best_estimator_.get_params()["subsample"], end=", ")
    print("Max features at split: ", best_model.best_estimator_.get_params()["colsample_bytree"], end=", ")
    print("Min child weight: ",
          best_model.best_estimator_.get_params()["min_child_weight"], end=", ")
    print("Alpha: ", best_model.best_estimator_.get_params()["reg_alpha"], end=", ")
    print("Lambda: ", best_model.best_estimator_.get_params()["reg_lambda"], end=", ")
    print("Number of estimators (trees): ", best_model.best_estimator_.get_params()["n_estimators"], end="}\n\n")

    return best_model


In [4]:
def report_classifier_performance(model, x, y):
    predict = model.predict(x)
    predict_proba = model.predict_proba(x)[:, 1]
    print("Accuracy :", metrics.accuracy_score(y, predict))
    print("logloss:", metrics.log_loss(y, predict))
    print("AUC Score:", metrics.roc_auc_score(y, predict_proba))
    print()

    
# Classifier parameter grid
class_parameters = {"learning_rate": [0.1, 0.05, 0.01, 0.001], 
                    "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                    "max_depth": [4, 5, 6, 7, 9],
                    "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                    "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                    "min_child_weight": [1, 3, 5, 7],
                    "n_estimators": [100, 250, 500, 1000, 1500, 2000]}

In [5]:
# Baseline classifier
baseline_classifier = XGBClassifier(objective='binary:logistic', scale_pos_weight=1)
xgb_rscv_classifier = RandomizedSearchCV(baseline_classifier, param_distributions=class_parameters,
                                         scoring="neg_log_loss", verbose=False, cv=5, random_state=2, n_jobs=4, n_iter=50)

# Find the optimal classifier and its performance on training and test set respectively
for name in train_data["class"]:
    print("Prediction of " + name)
    model = find_best_model(xgb_rscv_classifier, train_data["class"][name]["x"], train_data["class"][name]["y"])
    #model = XGBClassifier(objective="binary:logistic", learning_rate=0.01, gamma=1, max_depth=10, seed=27, scale_pos_weight=1,
    #                      subsample=0.5, colsample_bytree=1.0, min_child_weight=1, reg_alpha=0, reg_lambda=1, n_estimators=100)
    #model.fit(train_data["class"][name]["x"], train_data["class"][name]["y"])
    print("Performance on training set:")
    report_classifier_performance(model, train_data["class"][name]["x"], train_data["class"][name]["y"])
    print("Performance on test set:")
    report_classifier_performance(model, test_data["class"][name]["x"], test_data["class"][name]["y"])
    print("-------------------------------------------------------------------------------------------------\n")

Prediction of panas_pos_raw_class_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


processing

Optimal parameters: {Learning rate:  0.001, Gamma:  0.01, Max depth:  5, Subsample:  0.4, Max features at split:  0.8, Min child weight:  1, Alpha:  0, Lambda:  1, Number of estimators (trees):  1500}

Performance on training set:
Accuracy : 0.9655172413793104
logloss: 1.1909922894796794
AUC Score: 0.9952380952380953

Performance on test set:
Accuracy : 0.7777777777777778
logloss: 7.675372487472395
AUC Score: 0.8333333333333334

-------------------------------------------------------------------------------------------------

Prediction of panas_neg_raw_class_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


processing

Optimal parameters: {Learning rate:  0.001, Gamma:  2, Max depth:  5, Subsample:  0.7, Max features at split:  0.6, Min child weight:  1, Alpha:  0, Lambda:  1, Number of estimators (trees):  500}

Performance on training set:
Accuracy : 0.9
logloss: 3.4539309459864147
AUC Score: 0.9819004524886878

Performance on test set:
Accuracy : 0.4444444444444444
logloss: 19.188386796601534
AUC Score: 0.35000000000000003

-------------------------------------------------------------------------------------------------

Prediction of flourishing_scale_raw_class_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


processing

Optimal parameters: {Learning rate:  0.05, Gamma:  1, Max depth:  4, Subsample:  0.4, Max features at split:  0.8, Min child weight:  1, Alpha:  0, Lambda:  1, Number of estimators (trees):  1000}

Performance on training set:
Accuracy : 0.9642857142857143
logloss: 1.233527728389668
AUC Score: 1.0

Performance on test set:
Accuracy : 0.5555555555555556
logloss: 15.350833819103704
AUC Score: 0.6499999999999999

-------------------------------------------------------------------------------------------------

Prediction of panas_pos_imp_class_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


processing

Optimal parameters: {Learning rate:  0.05, Gamma:  1, Max depth:  9, Subsample:  0.2, Max features at split:  0.8, Min child weight:  1, Alpha:  0, Lambda:  1, Number of estimators (trees):  1000}

Performance on training set:
Accuracy : 0.9333333333333333
logloss: 2.302638399489392
AUC Score: 0.9955357142857143

Performance on test set:
Accuracy : 0.6666666666666666
logloss: 11.513103153288048
AUC Score: 0.7777777777777779

-------------------------------------------------------------------------------------------------

Prediction of panas_neg_imp_class_post


In [4]:
def report_regression_performance(model, x, y):
    predict = model.predict(x)
    print("RMSE :", math.sqrt(metrics.mean_squared_error(y, predict)))
    df = pd.DataFrame({'Actual': np.array(y).reshape(-1),
                       'Predicted': np.array(predict).reshape(-1)})
    display(df)


# Classifier parameter grid
regr_parameters = {"colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "min_child_weight": [1, 3, 5, 7],
                   "learning_rate": [0.1, 0.05, 0.01, 0.001],
                   "max_depth": [2, 4, 7, 10],
                   "n_estimators": [100, 250, 500, 1000, 1500, 2000],
                   'reg_alpha': [1e-5, 1e-2,  0.75],
                   'reg_lambda': [1e-5, 1e-2, 0.45],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7]}                                           

In [5]:
# Baseline regressor
baseline_regressor = XGBRegressor(objective="reg:squarederror")
xgb_rscv_regressor = RandomizedSearchCV(baseline_regressor, param_distributions=regr_parameters, 
                                        scoring="neg_mean_squared_error", iid=False, 
                                        cv=5, verbose=False, n_jobs=4, n_iter=50)

# Find the optimal classifier and its performance on training and test set respectively
for name in train_data["regr"]:
    print("Prediction of " + name)
    model = find_best_model(xgb_rscv_regressor, train_data["regr"][name]["x"], train_data["regr"][name]["y"])
    print("Performance on training set:")
    report_regression_performance(model, train_data["regr"][name]["x"], train_data["regr"][name]["y"])
    print("Performance on test set:")
    report_regression_performance(model, test_data["regr"][name]["x"], test_data["regr"][name]["y"])
    print("-------------------------------------------------------------------------------------------------\n")

Prediction of panas_pos_raw_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


processing

Optimal parameters: {Learning rate:  0.05, Gamma:  0.5, Max depth:  4, Subsample:  0.7, Max features at split:  0.3, Min child weight:  7, Alpha:  0.01, Lambda:  0.45, Number of estimators (trees):  100}

Performance on training set:
RMSE : 2.3006675958787994


,Actual,Predicted
0,31.0,31.879549
1,27.0,27.206074
2,28.0,29.083790
3,34.0,33.861973
4,32.0,32.215164
5,43.0,36.408585
6,36.0,34.397022
7,33.0,32.161957
8,17.0,20.840256
9,27.0,27.204685


Performance on test set:
RMSE : 5.632913053882458


...

processing

Optimal parameters: {Learning rate:  0.05, Gamma:  0.5, Max depth:  10, Subsample:  0.5, Max features at split:  0.3, Min child weight:  7, Alpha:  0.75, Lambda:  1e-05, Number of estimators (trees):  500}

Performance on training set:
RMSE : 2.309563156087042


,Actual,Predicted
0,45.0,46.579205
1,46.0,45.342564
2,44.0,44.611248
3,33.0,33.381367
4,47.0,47.161240
5,39.0,39.527966
6,53.0,50.251637
7,48.0,47.564167
8,41.0,41.864029
9,38.0,38.043617


Performance on test set:
RMSE : 6.842601943910931


,Actual,Predicted
0,31.0,37.425507
1,31.0,42.945602
2,50.0,46.617569
3,42.0,46.930843
4,48.0,46.114059
5,47.0,40.178093
6,37.0,47.012836
7,49.0,46.030804
8,41.0,47.517975


-------------------------------------------------------------------------------------------------

